In [839]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [840]:
# input the data sets
df1 = pd.read_csv("C:\\Users\\Chase\\Downloads\\PD 2022 Week 33 Input Online Orders.csv")
df2 = pd.read_csv("C:\\Users\\Chase\\Downloads\\PD 2022 Week 33 Input Instore Orders.csv")
df3 = pd.read_csv("C:\\Users\\Chase\\Downloads\\Preppin' Summer 2022 - Product Lookup.csv")

In [841]:
# Link the Instore and Online sales together to be one data source
# Call the NaNs in the Stores field "Online".
from numpy import NaN


df = pd.concat([df1, df2])
df['Store'] = np.where(df['Store'].isna(), 'Online', df['Store'])
df['Sales Timestamp'] = pd.to_datetime(df['Sales Timestamp'])
df['Sales Date'] = pd.to_datetime(df['Sales Date'])
df['Sales Timestamp'] = np.where(df['Sales Timestamp'].isna(), df['Sales Date'], df['Sales Timestamp'])
pd.DataFrame(df)

,Sales Timestamp,ID,Product,Sales Date,Store
0,2022-01-01 01:27:00,1,2,NaT,Online
1,2022-01-01 02:01:00,2,3,NaT,Online
2,2022-01-01 02:54:00,3,2,NaT,Online
3,2022-01-01 04:19:00,4,5,NaT,Online
4,2022-01-01 05:54:00,5,9,NaT,Online
...,...,...,...,...,...
995,2022-01-18 12:00:00,496,7,2022-01-18 12:00:00,Lewisham
996,2022-01-18 12:00:00,497,9,2022-01-18 12:00:00,Lewisham
997,2022-01-18 12:00:00,498,3,2022-01-18 12:00:00,Lewisham
998,2022-01-18 13:00:00,499,3,2022-01-18 13:00:00,Lewisham


In [842]:
# Link in the product Lookup to name the products instead of having their ID number
df = df.merge(df3, how='left', left_on='Product', right_on='Product ID')
df = df[['Sales Timestamp', 'ID', 'Product Name', 'Store']]
df

,Sales Timestamp,ID,Product Name,Store
0,2022-01-01 01:27:00,1,Liquid - 50ml,Online
1,2022-01-01 02:01:00,2,Liquid - 100ml,Online
2,2022-01-01 02:54:00,3,Liquid - 50ml,Online
3,2022-01-01 04:19:00,4,Liquid - 500ml,Online
4,2022-01-01 05:54:00,5,Bar - 2x,Online
...,...,...,...,...
1995,2022-01-18 12:00:00,496,Liquid - 1L,Lewisham
1996,2022-01-18 12:00:00,497,Bar - 2x,Lewisham
1997,2022-01-18 12:00:00,498,Liquid - 100ml,Lewisham
1998,2022-01-18 13:00:00,499,Liquid - 100ml,Lewisham


In [843]:
# Create the "Product Type" field by taking the first word of the product name
df[['Product Type', 'Amount']] = df['Product Name'].str.split('-', expand= True)
df = df[['Sales Timestamp', 'ID', 'Store', 'Product Type']]
df = df.sort_values(by=['Store', 'Sales Timestamp'])
df

,Sales Timestamp,ID,Store,Product Type
1500,2022-01-01 10:00:00,1,Lewisham,Liquid
1501,2022-01-01 11:00:00,2,Lewisham,Liquid
1502,2022-01-01 11:00:00,3,Lewisham,Liquid
1512,2022-01-01 11:00:00,13,Lewisham,Liquid
1513,2022-01-01 12:00:00,14,Lewisham,Bar
...,...,...,...,...
1343,2022-12-01 16:00:00,344,Wimbledon,Bar
1333,2022-12-01 17:00:00,334,Wimbledon,Liquid
1334,2022-12-01 17:00:00,335,Wimbledon,Liquid
1344,2022-12-01 17:00:00,345,Wimbledon,Bar


In [844]:
# Create a data set from your work so far that includes the next sale after the one made in the store of the same product type.
df['Next Sale'] = df.groupby(['Store', 'Product Type'])['Sales Timestamp'].shift(-1)
df = df[['Sales Timestamp', 'Next Sale', 'Store', 'Product Type']]
df

,Sales Timestamp,Next Sale,Store,Product Type
1500,2022-01-01 10:00:00,2022-01-01 11:00:00,Lewisham,Liquid
1501,2022-01-01 11:00:00,2022-01-01 11:00:00,Lewisham,Liquid
1502,2022-01-01 11:00:00,2022-01-01 11:00:00,Lewisham,Liquid
1512,2022-01-01 11:00:00,2022-01-01 13:00:00,Lewisham,Liquid
1513,2022-01-01 12:00:00,2022-01-01 16:00:00,Lewisham,Bar
...,...,...,...,...
1343,2022-12-01 16:00:00,2022-12-01 17:00:00,Wimbledon,Bar
1333,2022-12-01 17:00:00,2022-12-01 17:00:00,Wimbledon,Liquid
1334,2022-12-01 17:00:00,2022-12-01 17:00:00,Wimbledon,Liquid
1344,2022-12-01 17:00:00,NaT,Wimbledon,Bar


In [845]:
# Workout how long it took between the original sale and the next sale in minutes
# Remove any negative differences. There are sales that got refunded
df['Sales Timestamp'] = pd.to_datetime(df['Sales Timestamp'])
df['Next Sale'] = pd.to_datetime(df['Next Sale'])
df['Mins to next sale'] = round((df['Next Sale'] - df['Sales Timestamp']).dt.total_seconds() / 60, 1)
df = df.drop(df[df['Mins to next sale'] <= 0].index)
df = df.drop(df[(df['Mins to next sale'] >= 18000)].index)
df

,Sales Timestamp,Next Sale,Store,Product Type,Mins to next sale
1500,2022-01-01 10:00:00,2022-01-01 11:00:00,Lewisham,Liquid,60.0
1512,2022-01-01 11:00:00,2022-01-01 13:00:00,Lewisham,Liquid,120.0
1513,2022-01-01 12:00:00,2022-01-01 16:00:00,Lewisham,Bar,240.0
1503,2022-01-01 13:00:00,2022-01-01 14:00:00,Lewisham,Liquid,60.0
1514,2022-01-01 14:00:00,2022-01-01 15:00:00,Lewisham,Liquid,60.0
...,...,...,...,...,...
1351,2022-12-01 15:00:00,2022-12-01 16:00:00,Wimbledon,Bar,60.0
1342,2022-12-01 16:00:00,2022-12-01 17:00:00,Wimbledon,Liquid,60.0
1343,2022-12-01 16:00:00,2022-12-01 17:00:00,Wimbledon,Bar,60.0
1344,2022-12-01 17:00:00,NaT,Wimbledon,Bar,NaN


In [846]:
# Create a data set that shows the average of these values for each store and product type. Call this field "Average mins to next sale" 
df = df.groupby(['Store', 'Product Type'])['Mins to next sale'].mean()
df = pd.DataFrame(df)
df = df.reset_index()
df = df.rename(columns={'Mins to next sale': 'Average mins to next sale'})
df['Average mins to next sale'] = round(df['Average mins to next sale'], 1)
df

,Store,Product Type,Average mins to next sale
0,Lewisham,Bar,331.0
1,Lewisham,Liquid,218.1
2,Online,Bar,204.8
3,Online,Liquid,96.6
4,Wimbledon,Bar,326.8
5,Wimbledon,Liquid,214.0
